## Code for reproducing analyses from "Zhang & Yu, 2024"

### EEG experiment

In [ ]:
import mne, glob, os, mne_rsa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import product
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score,KFold
import pingouin as pg
from mne.stats import permutation_cluster_1samp_test
from joblib import Parallel, delayed
from matplotlib import rc
from shapely.geometry import Polygon
from sklearn.decomposition import PCA
import copy
from sklearn.model_selection import train_test_split
from matplotlib import font_manager
from sklearn.utils import resample
from mne import io
from mne_connectivity import spectral_connectivity_epochs, seed_target_indices, spectral_connectivity_time
from mne.time_frequency import AverageTFR, tfr_morlet
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mne.stats import spatio_temporal_cluster_test, combine_adjacency, spatio_temporal_cluster_1samp_test
from mne.channels import find_ch_adjacency
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rcParams['font.family']= 'sans-serif'
plt.rcParams['font.sans-serif']= 'Helvetica'

In [ ]:
#read in data
epochs=[]
metadata=[]
for i in range(22):
    epochs.append(mne.read_epochs('s0%s_eeg_cleaned-epo.fif.gz'%(i+1)))
    metadata.append(epochs[-1].metedata)

In [ ]:
#define frontal middle and posterior channels
picks_f=[a for a in reconst_epochs_p2.ch_names if 'F' in a and 'C' not in a and 'T' not in a]
picks_p=[a for a in reconst_epochs_p2.ch_names if 'P' in a and  'T' not in a and 'C' not in a or 'O' in a]
picks_m = [a for a in reconst_epochs_p2.ch_names if 'C' in a or 'T' in a]

In [ ]:
def calculate_null(rdms, use_pca=True):
    temp=[]
    for ss, rdm_arr in enumerate(rdms):
        circularity=[]
        for i in range(len(w_start[:-1])): 
            scaler = StandardScaler()
            embedding =  PCA(n_components=2)
            if use_pca:
                coor = pca_ind[ss][i].transform(scaler.fit_transform(rdm_arr[i]))
            else:
                coor = embedding.fit_transform(scaler.fit_transform(rdm_arr[i]))
            c1x =coor[:,0]
            c1y =coor[:,1]
            polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[2],c1y[2]),(c1x[0],c1y[0])))
            circularity.append(4*np.pi*polygon.area/(polygon.length)**2)
        temp.append(circularity)      
    return temp

In [ ]:
def shuffle_trial_label(picks):
    resampled_rdm=[]
    for t in range(5):
        rdms_ss_v=[]   
        for i, f in enumerate(epochs):
            m = metadata[i]
            m['trial_type'] = 0 #recode trials into trial_types based on task feature of interest
            m['target_size'] = np.where(m.answer_size.values<0.19, 1,np.where(m.answer_size.values>0.26, 3, 2))
            m['target_color'] = np.where(m.answer_color.values<42, 1,np.where(m.answer_color.values>74, 3, 2))
            x_train = f.copy().pick(picks).get_data()#select channels
            # for j,l in enumerate(list(product((1,2,3),(1,2,3)))): #for stimulus space
            #     m.loc[(m['whichSize']==l[0])&(m['whichColor']==l[1]), 'trial_type'] = j
            # for j,l in enumerate(list(product((1,2,3),(1,2,3)))): #for target stimulus space
            #     m.loc[(m['target_size']==l[0])&(m['target_color']==l[1]), 'trial_type'] = j
            y_train =  m.cue.values -1 #for goal space
            # y_train = m['trial_type'].values # for stimulus or target spaces
            if t!=0:
                x_train, y_train= resample(x_train, y_train, stratify=y_train)
            n_trials, n_sensors, n_times= x_train.shape 
            eeg_data =np.zeros((np.unique(y_train).shape[0], n_sensors, n_times))
            np.random.shuffle(y_train) #shuffle trial labels
            for i in range(np.unique(y_train).shape[0]):
                eeg_data[i,:,:] = np.mean(x_train[y_train==i],axis=0)
            w_start = np.where(np.isin(f.times, f.times[0::20])==True)[0]
            rdms = []
            for i in range(len(w_start[:-1])): 
                rdms.append(np.mean(eeg_data[:, :, w_start[i]:w_start[i+1]], axis=2))
            rdms_ss_v.append(rdms)
        resampled_rdm.append(rdms_ss_v)
    return resampled_rdm

In [ ]:
def cluster_statistic():
    ci_null = np.array(shuffled_ci).mean(1)[np.random.choice(np.arange(np.array(shuffled_ci).mean(1).shape[0]))]
    pvalues_null=np.zeros(len(w_start[:-1]))
    for j in range(pvalues.shape[0]):
        pvalues_null[j] = np.where(np.array(shuffled_ci).mean(1)[:,j]>ci_null[j])[0].shape[0]/len(np.array(shuffled_ci).mean(1))

    d = np.where(pvalues_null<0.05)[0] #get clusters bigger than 0
    a = np.split(d,np.where(np.diff(d)!=1)[0]+1) #get consecutive points
    return np.max([len(x) for x in a]) #count location only

In [ ]:
def permute_label_splittrial(picks,stimulus=False, exclude_trial=False):
    scaler = MinMaxScaler() #0-1 range
    rdms_ss_good, rdms_ss_bad=[],[]   
    for i, f in enumerate(epochs):
        x_train = f.copy().pick(picks).get_data()#select channels
        
        m = metadata[i]#recode trials into trial_types based on all features
        m['trial_type'] = 0
        m['response'] = 0
        m['error_color'] =  (m.answer_color-m.correct_color) 
        m['error_size'] = (m.answer_size-m.correct_size)
        m['error_size_per'] = m.error_size/m['size'] 
        m['error_color'] =  scaler.fit_transform(m.error_color.values.reshape(-1,1))
        m['error_size_per'] =  scaler.fit_transform(m.error_size_per.values.reshape(-1,1))
        total_error =m['error_color'].values+m['error_size_per'].values
        for j,l in enumerate(list(product((1,2,3),(1,2,3)))): #test stimulus space
            m.loc[(m['whichSize']==l[0])&(m['whichColor']==l[1]), 'trial_type'] = j

        #using absolute error values
        if stimulus==True :
            ind_goodtrial = m.loc[(np.percentile(total_error, 50)> total_error) ].index.values #half-half
            ind_badtrial = m.loc[(np.percentile(total_error,50)<= total_error) ].index.values
        else:
            ind_goodtrial = m.loc[(np.percentile(total_error, 25)> total_error) ].index.values #upper and lower quantile
            ind_badtrial = m.loc[(np.percentile(total_error,75)<= total_error) ].index.values
            if i==10: #one subject doesnt have enough trials so loosen the threshold a bit
                ind_goodtrial = m.loc[(np.percentile(total_error, 25)> total_error) ].index.values
                ind_badtrial = m.loc[(np.percentile(total_error, 70)<= total_error) ].index.values

        y_train = m.cue.values-1 #for goal space
        if stimulus==True: #for stimulus space
            y_train = m['trial_type'].values

        #split trials based on error magnitude
        xtrain_badtrial = x_train[ind_badtrial]
        xtrain_goodtrial = x_train[ind_goodtrial]
        ytrain_badtrial = y_train[ind_badtrial]
        ytrain_goodtrial = y_train[ind_goodtrial]
        
        #randomly assign membership between the good and bad trials
        x1,x2, y1,y2 =train_test_split(np.concatenate([xtrain_badtrial,xtrain_goodtrial]), np.concatenate([ytrain_badtrial,ytrain_goodtrial]), \
                                       test_size=0.5, train_size=0.5, stratify=np.concatenate([ytrain_badtrial,ytrain_goodtrial]))
        n_trials, n_sensors, n_times= x_train.data.shape 
        eeg_data_badtrial =np.zeros((np.unique(y_train).shape[0], n_sensors, n_times))
        eeg_data_goodtrial =np.zeros((np.unique(y_train).shape[0], n_sensors, n_times))
        for i in range(np.unique(y_train).shape[0]): #average within conditions
            eeg_data_badtrial[i,:,:] = np.mean(x1[y1==i],axis=0)
            eeg_data_goodtrial[i,:,:] = np.mean(x2[y2==i],axis=0)
        w_start = np.where(np.isin(f.times, f.times[0::20])==True)[0]
        rdms_bad,rdms_good = [],[]
        for n in range(len(w_start[:-1])): 
            rdms_bad.append(np.mean(eeg_data_badtrial[:, :, w_start[n]:w_start[n+1]], axis=2))
            rdms_good.append(np.mean(eeg_data_goodtrial[:, :, w_start[n]:w_start[n+1]], axis=2))
        rdms_ss_good.append(rdms_good)
        rdms_ss_bad.append(rdms_bad)
    return rdms_ss_good, rdms_ss_bad

In [ ]:
def shuffle_label_splittrial(picks, stimulus=False):
    scaler = MinMaxScaler() #0-1 range
    rdms_ss_good, rdms_ss_bad=[],[]   
    for i, f in enumerate(epochs):
        x_train = f.copy().pick(picks).get_data()#select channels
        #recode trials into trial_types based on all features
        m = metadata[i]
        m['trial_type'] = 0
        m['response'] = 0

        #exclude large error trials
        m['error_color'] =  (m.answer_color-m.correct_color) 
        m['error_size'] = (m.answer_size-m.correct_size)
        m['error_size_per'] = m.error_size/m['size'] 
        m['error_color'] =  scaler.fit_transform(m.error_color.values.reshape(-1,1))
        m['error_size_per'] =  scaler.fit_transform(m.error_size_per.values.reshape(-1,1))
        total_error =m['error_color'].values+m['error_size_per'].values
        for j,l in enumerate(list(product((1,2,3),(1,2,3)))): #test stimulus space
            m.loc[(m['whichSize']==l[0])&(m['whichColor']==l[1]), 'trial_type'] = j
        m['target_size'] = np.where(m.answer_size.values<0.19, 1,np.where(m.answer_size.values>0.26, 3, 2))
        m['target_color'] = np.where(m.answer_color.values<42, 1,np.where(m.answer_color.values>74, 3, 2))
        for j,l in enumerate(list(product((1,2,3),(1,2,3)))): #test stimulus space
            m.loc[(m['target_size']==l[0])&(m['target_color']==l[1]), 'response'] = j
        
        #using absolute error values
        if stimulus==True or stimulus=='response':
            ind_goodtrial = m.loc[(np.percentile(total_error, 50)> total_error) ].index.values
            ind_badtrial = m.loc[(np.percentile(total_error,50)<= total_error) ].index.values
        else:
            ind_goodtrial = m.loc[(np.percentile(total_error, 25)> total_error) ].index.values
            ind_badtrial = m.loc[(np.percentile(total_error,75)<= total_error) ].index.values
            if i==10:
                ind_goodtrial = m.loc[(np.percentile(total_error, 25)> total_error) ].index.values
                ind_badtrial = m.loc[(np.percentile(total_error, 70)<= total_error) ].index.values

        y_train = m.cue.values-1
        if stimulus==True:
            y_train = m['trial_type'].values
        elif stimulus=='response':
            y_train = m['response'].values
        n_trials, n_sensors, n_times= x_train.data.shape 

        #split half
        xtrain_badtrial = x_train[ind_badtrial]
        xtrain_goodtrial = x_train[ind_goodtrial]
        ytrain_badtrial = y_train[ind_badtrial]
        ytrain_goodtrial = y_train[ind_goodtrial]

        #if we are not resampling just using real data
        x_badtrial,y_badtrial=  xtrain_badtrial , ytrain_badtrial
        x_goodtrial,y_goodtrial = xtrain_goodtrial, ytrain_goodtrial

        eeg_data_badtrial =np.zeros((np.unique(y_train).shape[0], n_sensors, n_times))
        eeg_data_goodtrial =np.zeros((np.unique(y_train).shape[0], n_sensors, n_times))
        for i in range(np.unique(y_train).shape[0]):
            eeg_data_badtrial[i,:,:] = np.mean(x_badtrial[y_badtrial==i],axis=0)
            eeg_data_goodtrial[i,:,:] = np.mean(x_goodtrial[y_goodtrial==i],axis=0)
        w_start = np.where(np.isin(f.times, f.times[0::20])==True)[0]#if want to draw time course
        rdms_bad,rdms_good = [],[]
        for n in range(len(w_start[:-1])): 
            rdms_bad.append(np.mean(eeg_data_badtrial[:, :, w_start[n]:w_start[n+1]], axis=2)) #not using similarity matrix just use raw data
            rdms_good.append(np.mean(eeg_data_goodtrial[:, :, w_start[n]:w_start[n+1]], axis=2))
        rdms_ss_good.append(rdms_good)
        rdms_ss_bad.append(rdms_bad)
    return rdms_ss_good, rdms_ss_bad

In [ ]:
def run_null(rdms, ss, pcas, scalers):
    temp=[]
    for i in range(len(w_start[:-1])):
        scaler=StandardScaler()
        pca= PCA(2)
        coor = pcas[i].transform(scaler.fit_transform(rdms[i])) #using the original pca and scaler trained by entire dataset
        # coor = pcas[i].transform(scaler.fit_transform(np.delete(rdms[i], [1,4,7],axis=0))) 
        c1x =coor[:,0]
        c1y =coor[:,1]
        polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[2],c1y[2]),(c1x[0],c1y[0])))
        # polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[5],c1y[5]),(c1x[4],c1y[4]),(c1x[2],c1y[2]),(c1x[0],c1y[0])))
        temp.append(4*np.pi*polygon.area/(polygon.length)**2)  
    return temp

In [ ]:
#Figure 1B
#prepare data; We resample the raw data a few times to get rid of randomness
resampled_rdms=[]
for t in range(5):
    rdms_ss_v=[]   
    scaler=StandardScaler()
    for i, f in enumerate(epochs):
        m = metadata[i]
        x_train = f.copy().pick(picks_f).get_data()#select channels
        y_train =  m.cue.values -1  #for plotting goal circularity
        n_trials, n_sensors, n_times= x_train.shape 
        if t!=0:
            x_train, y_train= resample(x_train, y_train, stratify=y_train)
        eeg_data =np.zeros((np.unique(y_train).shape[0], n_sensors, n_times))# eeg_data 1st dim matches num of features in conceptual model
        for k in range(np.unique(y_train).shape[0]):
            eeg_data[k,:,:] = np.mean(x_train[y_train==k],axis=0)
        w_start = np.where(np.isin(f.times, f.times[0::20])==True)[0]#if want to draw time course, we use sliding window of 80 ms long
        rdms = []
        for n in range(len(w_start[:-1])): 
            rdms.append(np.mean(eeg_data[:, :, w_start[n]:w_start[n+1]], axis=2)) 
        rdms_ss_v.append(rdms)
    resampled_rdms.append(rdms_ss_v)    
    
#calculate real individual circularity
resampled_ci=[]
for t in range(5):
    temp=[]
    if t==0:
        pca_ind , scaler_ind= [],[]
    for ss, rdm_arr in enumerate(resampled_rdms[t]):
        m= metadata[ss]
        circularity=[]
        temp1,temp2=[],[]
        for i in range(len(w_start[:-1])):
            scaler = StandardScaler()
            embedding =  PCA(n_components=2)
            coor = embedding.fit_transform(scaler.fit_transform(rdm_arr[i]))
            temp1.append(embedding);temp2.append(scaler)
            c1x =coor[:,0]
            c1y =coor[:,1]
            polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[2],c1y[2]),(c1x[0],c1y[0])))
            circularity.append(4*np.pi*polygon.area/(polygon.length)**2)
        if t==0:
            pca_ind.append(temp1);scaler_ind.append(temp2)
        temp.append(circularity) 
    resampled_ci.append(temp)
    
individual_ci=[]
for t in range(len(w_start[:-1])):
    individual_ci.append(np.array(resampled_ci).mean(0)[:,t])
individual_ci=np.array(individual_ci).T

#get distribution of shuffled data matrices
#Can take 5-8 hours depending on the CPU
#Note that to create distribution of different circularity indices (goals vs. stimulus), you need to manually change the y_train in the function
shuffled_rdm_ss= Parallel(n_jobs=25)(delayed(shuffle_trial_label)(picks_f) for i in range(5000))


#calculating individual circularity using the shuffled data 
w_start = np.where(np.isin(f.times, f.times[0::20])==True)[0]#if want to draw time course
shuffled_ci_resampled =[]
for t in range(5):
    #use Parallel to speed up
    shuffled_ci = Parallel(n_jobs=25)(delayed(calculate_null)(rdms,use_pca=t==0) for rdms in np.array(shuffled_rdm_ss)[:,t])
    shuffled_ci_resampled.append(shuffled_ci)
shuffled_ci = np.array(shuffled_ci_resampled).mean(0) #if resampling trials first averaging over resmapled folds

#manual cluster-level statistics
cluster_perm_total=  Parallel(n_jobs=20)(delayed(cluster_statistic)() for i in range(5000))
threshold_total = np.percentile(cluster_perm_total, 95, method='closest_observation')
coor=[]
for x in a:
    if len(x)>threshold_total:
        coor.extend(x) 
    
#Plotting
fig, ax =plt.subplots(1,1,figsize=(6,3.))
data = individual_ci
plt.plot(f.times[w_start[:-1]+20], data.mean(0), c='orange', label='difference')
plt.fill_between(f.times[w_start[:-1]+20],data.mean(0)+ data.std(0)/np.sqrt(data.shape[0]),data.mean(0)- data.std(0)/np.sqrt(data.shape[0]),alpha=0.2,color='orange')
plt.scatter(np.take(f.times[w_start[:-1]+20], coor), [0.25]*len(coor),c='r',marker='_')#highlight significant period
ax.axvline(0,c='k')
ax.axvspan(0.4,1.7, alpha=.15,color='gray')
ax.axvspan(2.3,3.8, alpha=.15,color='gray')
ax.set_ylabel('Individual circularity',fontsize=16)
ax.set_xlabel('Time (s)',fontsize=15)
ax.set_yticks([0.2,0.3,0.4])
ax.set_yticklabels([0.2,0.3,0.4],fontsize=15, va='center')
ax.set_xticks([0,0.4,1.7,2.3,3.8])
ax.set_xticklabels(['0',0.4,1.7,2.3,3.8] ,fontsize=15, va='center')
ax.tick_params(bottom=True)
ax.xaxis.set_tick_params(pad=10)
for y in [0.4,1.7,2.3,3.8]:
    ax.axvline(y, linestyle='--', color='k', linewidth=0.8)
sns.despine(fig=fig, right=True, top=True)
plt.tight_layout()

In [ ]:
#Figure 1C
#prepare data matrix
rdms_ss_v=[]   
for i, f in enumerate(epochs):
    m = metadata[i]
    x_train = f.copy().pick(picks_f).get_data()#select channels
    y_train =  m.cue.values -1  #for plotting goal circularity
    n_trials, n_sensors, n_times= x_train.shape 
    eeg_data =np.zeros((np.unique(y_train).shape[0], n_sensors, n_times))# eeg_data 1st dim matches num of features in conceptual model
    for k in range(np.unique(y_train).shape[0]):
        eeg_data[k,:,:] = np.mean(x_train[y_train==k],axis=0)#average within conditions
    w_start = np.where(np.isin(f.times, [-0.5,0,0.4,1.7,2.3,3.8,4.3])==True)[0]  #select the time windows corresponding to task epochs
    rdms = []
    for n in range(len(w_start[:-1])): 
        rdms.append(np.mean(eeg_data[:, :, w_start[n]:w_start[n+1]], axis=2)) #average within time windows
    rdms_ss_v.append(rdms)

#perform PCA
dfs_mds=[]
categorical_model = [1,2,3,4]
for i in range(len(w_start[:-1])):
    scaler = StandardScaler()
    a = [x[i] for x in rdms_ss_v] #concatenate subjects 
    embedding = PCA(n_components=3)
    coor = embedding.fit_transform(scaler.fit_transform(np.hstack(a))) #standardize again before applying PCA
    df = pd.DataFrame(coor, columns=['PC1','PC2','PC3'])
    df['cue'] = categorical_model 
    df['time']=i
    dfs_mds.append(df)
dfs_mds = pd.concat(dfs_mds, ignore_index=True)
    
#Plotting
palettes = ['yellow','green','r','darkblue']
axs=axs.ravel()
circularity_xy=[]
titles=['Pre-cue','Goal cue','Delay 1','Sample','Delay 2','Response']
dim1='PC1'; dim2='PC2'
for i in range(len(w_start[:-1])):
    data=dfs_mds[dfs_mds.time==i].groupby(['cue']).mean().reset_index()
    c1x =data[dim1].values
    c1y =data[dim2].values
    axs[i].plot([c1x[0],c1x[1],c1x[3],c1x[2],c1x[0]],[c1y[0],c1y[1],c1y[3],c1y[2],c1y[0]],lw=1.4, c='gray', zorder=1)
    sns.scatterplot(x=dim1, y=dim2, data=data, hue= 'cue',legend=False, ax=axs[i], palette=palettes, s=130,edgecolor='black',zorder=2, )
    axs[i].set_title(titles[i], fontsize=22)
    axs[i].set_xlabel('PC1',fontsize=16)
    axs[i].set_ylabel('PC2',fontsize=16)
    axs[i].set(xticklabels=[],yticklabels=[])
    axs[i].tick_params(bottom=False, left=False)
    polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[2],c1y[2]),(c1x[0],c1y[0])))
    circularity_xy.append(4*np.pi*polygon.area/(polygon.length)**2) #calculate the circularity index

sns.despine(fig=fig, right=True, top=True)
plt.tight_layout(rect=[0, 0, 0.9, 1])

In [ ]:
#Figure 2A
#prepare data matrix
rdms_ss_v=[]   
for i, f in enumerate(epochs):
    m = metadata[i]
    x_train = f.copy().pick(picks_f).get_data()#select channels
    y_train =  m.cue.values -1  #for plotting goal circularity
    n_trials, n_sensors, n_times= x_train.shape 
    eeg_data =np.zeros((np.unique(y_train).shape[0], n_sensors, n_times))# eeg_data 1st dim matches num of features in conceptual model
    for k in range(np.unique(y_train).shape[0]):
        eeg_data[k,:,:] = np.mean(x_train[y_train==k],axis=0)#average within conditions
    w_start = np.where(np.isin(f.times, [-0.5,0,0.4,1.7,2.3,3.8,4.3])==True)[0]  #select the time windows corresponding to task epochs
    rdms = []
    for n in range(len(w_start[:-1])): 
        rdms.append(np.mean(eeg_data[:, :, w_start[n]:w_start[n+1]], axis=2)) #average within time windows
    rdms_ss_v.append(rdms)

dfs_mds_ss =[]
categorical_model = np.array([1,2,3,4]) 
for ss, rdm_arr in enumerate(rdms_ss_v):
    m= metadata[ss]
    m['error_color'] =  (m.answer_color-m.correct_color) #no within subject scaling, do it across subjects
    m['error_size'] = (m.answer_size-m.correct_size)
    m['error_size_per'] = m.error_size/m['size'] #size is a percentile construt
    a =m.shape[0]
    for i in range(len(w_start[:-1])):
        scaler = StandardScaler()
        embedding =  PCA(n_components=3)
        coor = embedding.fit_transform(scaler.fit_transform(rdm_arr[i]))
        df = pd.DataFrame(np.hstack((coor,categorical_model.reshape(-1,1))), columns=[0,1,2,'cue'])
        df['time']=i
        df['subject']=ss+1
        df['error_color'] = m.error_color.abs().mean()
        df['error_size'] = m.error_size.abs().mean()
        df['error_size_per'] = m.error_size_per.abs().mean()
        dfs_mds_ss.append(df)

scaler = MinMaxScaler()      
dfs_mds_ss = pd.concat(dfs_mds_ss, ignore_index=True) 
#scale the error terms across subjects before adding up for total error
dfs_mds_ss['error_color'] =  scaler.fit_transform(dfs_mds_ss.error_color.values.reshape(-1,1))
dfs_mds_ss['error_size'] =  scaler.fit_transform(dfs_mds_ss.error_size.values.reshape(-1,1))
dfs_mds_ss['error_size_per'] =  scaler.fit_transform(dfs_mds_ss.error_size_per.values.reshape(-1,1))
dfs_mds_ss['error_total'] = dfs_mds_ss['error_color']+dfs_mds_ss['error_size_per']

for ss in range(len(rdms_ss_v)):
    for i in range(len(w_start[:-1])):
        for dims in list(itertools.combinations((0,1,2),2)):
            dim1=dims[0]
            dim2=dims[1]
            c1x =dfs_mds_ss[(dfs_mds_ss.subject==ss+1)&(dfs_mds_ss.time==i)][dim1].values
            c1y =dfs_mds_ss[(dfs_mds_ss.subject==ss+1)&(dfs_mds_ss.time==i)][dim2].values
            polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[2],c1y[2]),(c1x[0],c1y[0])))
            dfs_mds_ss.loc[(dfs_mds_ss.subject==ss+1)&(dfs_mds_ss.time==i), 'ci_%s%s'%(dim1,dim2)] = 4*np.pi*polygon.area/(polygon.length)**2

#Plotting
fig, ax=plt.subplots(1,1,figsize=(5.5,4))
ax.tick_params(left=False, bottom=False)
sns.regplot( dfs_mds_ss.loc[dfs_mds_ss.time==2].groupby(['subject'])['ci_%s%s'%(dims[0],dims[1])].mean(),
               dfs_mds_ss.loc[dfs_mds_ss.time==2].groupby(['subject']).error_total.mean())
ax.set_ylabel('Standardized response error', fontsize=16)
ax.set_xlabel('Circularity index', fontsize=16)
ax.set_xticks([0,0.2,0.4,0.6])
ax.set_xticklabels([0,0.2,0.4,0.6],rotation=0 ,fontsize=16)
ax.set_yticks([0,0.5,1,1.5])
ax.set_yticklabels([0,0.5,1,1.5],rotation=0 ,fontsize=16)
sns.despine(fig=fig,top=True, right=True)
ax.text(0.05,1.5, r'$r = 0.17, p = 0.78$',fontsize=18)

In [ ]:
#Figure 3B-D
#get shuffled null distribution of good and bad trial 
#take ~3 hours
temps= Parallel(n_jobs=30)(delayed(permute_label_splittrial)(picks_f,stimulus=False) for i in range(5000))#for stimulus space, set stimulus parameter to True
shuffled_rdm_ss_good = [items[0] for items in temps]
shuffled_rdm_ss_bad = [items[1] for items in temps]

#calculate circularity difference 
trial_cond = ['good','bad']
w_start = np.where(np.isin(f.times, f.times[0::20])==True)[0]#if want to draw time course
dimensions=[]
for k,rdms_ss in enumerate([shuffled_rdm_ss_good,shuffled_rdm_ss_bad]):
    temp=[]
    for ss in range(len(metadata)):
        temp1 = Parallel(n_jobs=25)(delayed(run_null)(rdms_ss[n][ss], ss,  pca_ind[ss], scaler_ind[ss]) for n in range(len(rdms_ss)))
        temp.append(temp1)
    dimensions.append(np.mean(temp,0))
data = np.array(dimensions[0])-np.array(dimensions[1])

#calculate ciruclarity differnce for real data split
rdms_ss_good, rdms_ss_bad = shuffle_label_splittrial(picks_f,stimulus=False)                
dimensions=[]
for k,rdms_ss in enumerate([rdms_ss_good, rdms_ss_bad]):
    temp=[]
    for ss in range(len(metadata)):
        temp1=[]
        for i in range(len(w_start[:-1])):
            pca= PCA(2)
            coor = pca_ind[ss][i].transform(scaler_ind[ss][i].transform(rdms_ss[ss][i])) #standardize again before applying PCA
            c1x =coor[:,0]
            c1y =coor[:,1]
            polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[2],c1y[2]),(c1x[0],c1y[0]))) #for goal space
            # polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[5],c1y[5]),(c1x[4],c1y[4]),(c1x[2],c1y[2]),(c1x[0],c1y[0]))) #for stimulus space
            temp1.append(4*np.pi*polygon.area/(polygon.length)**2)  
        temp.append(temp1)
    dimensions.append(np.array(temp))
real_data = np.array(dimensions[0])-np.array(dimensions[1])

#plotting
fig, ax =plt.subplots(1,1,figsize=(6,3.))
data_dif=real_data
plt.plot(f.times[w_start[:-1]+20], data_dif.mean(0), c='orange', label='difference')
plt.fill_between(f.times[w_start[:-1]+20],data_dif.mean(0)+ data_dif.std(0)/np.sqrt(data_dif.shape[0]),data_dif.mean(0)- data_dif.std(0)/np.sqrt(data_dif.shape[0]),alpha=0.2,color='orange')
plt.axhline(0,c='k',linewidth=0.8)
for y in [0,0.4,1.7,2.3,3.8]:
    plt.axvline(y, linestyle='--', color='k', linewidth=0.8)
try:
    #if plot significant period too, need to use the cluster_statistic() function to find time points first
    plt.scatter(np.take(f.times[w_start[:-1]+20],coor), [-.05]*len(coor),c='r',marker='_')
except:
    pass
ax.axvline(0,c='k')
ax.axhline(0,c='k',linewidth=0.5)
ax.axvspan(0.4,1.7, alpha=.15,color='gray')
ax.axvspan(2.3,3.8, alpha=.15,color='gray')
ax.set_ylabel('Circularity difference',fontsize=16)
ax.set_xlabel('Time (s)',fontsize=15)
ax.set_yticks([-0.1,0,0.1,0.2])
ax.set_yticklabels([-0.1,0,0.1,0.2],fontsize=15, va='center')
ax.set_xticks([0,0.4,1.7,2.3,3.8])
ax.set_xticklabels(['0',0.4,1.7,2.3,3.8] ,fontsize=15, va='center')
ax.tick_params(bottom=True)
ax.xaxis.set_tick_params(pad=10)
for y in [0.4,1.7,2.3,3.8]:
    ax.axvline(y, linestyle='--', color='k', linewidth=0.8)
sns.despine(fig=fig, right=True, top=True)
plt.tight_layout()

In [ ]:
#Figure 4A
#Note that for response circularity, it is the same codes but need to recode conditions based on response values
resampled_rdms=[]
for t in range(5):
    rdms_ss_v=[]   
    for i, f in enumerate(epochs):
        m = metadata[i]
        m['trial_type'] = 0#recode trials into trial_types based on all features
        x_train = f.copy().pick(picks_p).get_data()#select channels
        for j,l in enumerate(list(product((1,2,3),(1,2,3)))): #test stimulus space
            m.loc[(m['whichSize']==l[0])&(m['whichColor']==l[1]), 'trial_type'] = j
        y_train = m['trial_type'].values #for plotting stimulus circularity
        if t!=0:
            x_train, y_train= resample(x_train, y_train, stratify=y_train)
        n_trials, n_sensors, n_times= x_train.shape 
        eeg_data =np.zeros((np.unique(y_train).shape[0], n_sensors, n_times))
        for k in range(np.unique(y_train).shape[0]):
            eeg_data[k,:,:] = np.mean(x_train[y_train==k],axis=0)
        w_start = np.where(np.isin(f.times, f.times[0::20])==True)[0]#if want to draw time course
        rdms = []
        for n in range(len(w_start[:-1])): 
            rdms.append(np.mean(eeg_data[:, :, w_start[n]:w_start[n+1]], axis=2)) 
        rdms_ss_v.append(rdms)
    resampled_rdms.append(rdms_ss_v)

#Create a null distribution specific to stimulus conditions (you need to manually change the code inside the function first)
shuffled_rdm_ss= Parallel(n_jobs=25)(delayed(shuffle_trial_label)(picks_p) for i in range(5000))

#calculate real individual circularity
resampled_ci_stim=[]
for t in range(5):
    temp=[]
    if t==0:
        pca_stim_ind , scaler_stim_ind= [],[]
    for ss, rdm_arr in enumerate(resampled_rdms[t]):
        m= metadata[ss]
        circularity=[]
        temp1,temp2=[],[]
        for i in range(len(w_start[:-1])):
            scaler = StandardScaler()
            embedding=PCA(n_components=2)
            coor = embedding.fit_transform(scaler.fit_transform(np.delete(rdm_arr[i],[1,4,7],axis=0))) #Discard the middle 3 conditions
            c1x =coor[:,0]
            c1y =coor[:,1]
            temp1.append(embedding);temp2.append(scaler)
            polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[5],c1y[5]),(c1x[4],c1y[4]),(c1x[2],c1y[2]),(c1x[0],c1y[0]))) #note this order is critical and can change
            circularity.append(4*np.pi*polygon.area/(polygon.length)**2)
        if t==0:
            pca_stim_ind.append(temp1);scaler_stim_ind.append(temp2)
        temp.append(circularity)  
    resampled_ci_stim.append(temp)
individual_ci_stim=[]
for t in range(len(w_start[:-1])):
    individual_ci_stim.append(np.array(resampled_ci_stim).mean(0)[:,t])
individual_ci_stim=np.array(individual_ci_stim).T

shuffled_ci_resampled =[]
for t in range(5):
    shuffled_ci = Parallel(n_jobs=25)(delayed( calculate_null)(rdms) for rdms in np.array(shuffled_rdm_ss)[:,t])
    shuffled_ci_resampled.append(shuffled_ci)
shuffled_ci_stim = np.array(shuffled_ci_resampled).mean(0)

#manual cluster level statistic
cluster_perm_total=  Parallel(n_jobs=25)(delayed(cluster_statistic)() for i in range(5000))
threshold_total = np.percentile(cluster_perm_total, 95, method='closest_observation')
coor=[]
for x in a:
    if len(x)>threshold_total:
        coor.extend(x) 

#Plotting
fig, ax =plt.subplots(1,1,figsize=(6,3.))
data = individual_ci_stim
plt.plot(f.times[w_start[:-1]+20], data.mean(0), c='orange', label='difference')
plt.fill_between(f.times[w_start[:-1]+20],data.mean(0)+ data.std(0)/np.sqrt(data.shape[0]),data.mean(0)- data.std(0)/np.sqrt(data.shape[0]),alpha=0.2,color='orange')
for y in [0,0.4,1.7,2.3,3.8]:
    plt.axvline(y, linestyle='--', color='k', linewidth=0.8)
try:
    plt.scatter(np.take(f.times[w_start[:-1]+20], coor), [0.15]*len(coor),c='r',marker='_')
except:
    pass
ax.axvline(0,c='k')
ax.axvspan(0.4,1.7, alpha=.15,color='gray')
ax.axvspan(2.3,3.8, alpha=.15,color='gray')
ax.set_ylim(0.12,0.25,auto=True)
ax.set_ylabel('Individual circularity',fontsize=15)
ax.set_xlabel('Time (s)',fontsize=15)
ax.set_yticks([0.15,0.2,0.25])
ax.set_yticklabels([0.15,0.2,0.25],fontsize=15, va='center')
ax.set_xticks([0,0.4,1.7,2.3,3.8])
ax.set_xticklabels(['0',0.4,1.7,2.3,3.8] ,fontsize=15, va='center')
ax.tick_params(bottom=True)
ax.xaxis.set_tick_params(pad=10)
for y in [0.4,1.7,2.3,3.8]:
    ax.axvline(y, linestyle='--', color='k', linewidth=0.8)
sns.despine(fig=fig, right=True, top=True)
plt.tight_layout()

In [ ]:
#Figure 4B
rdms_ss_v=[]   
for i, f in enumerate(epochs):
    m = metadata[i]
    m['trial_type'] = 0
    x_train = f.copy().pick(picks_p).get_data()#select channels
    for j,l in enumerate(list(product((1,2,3),(1,2,3)))): #test stimulus space
        m.loc[(m['whichSize']==l[0])&(m['whichColor']==l[1]), 'trial_type'] = j
    y_train = m['trial_type'].values #for plotting stimulus circularity
    n_trials, n_sensors, n_times= x_train.shape 
    eeg_data =np.zeros((np.unique(y_train).shape[0], n_sensors, n_times))# eeg_data 1st dim matches num of features in conceptual model
    for k in range(np.unique(y_train).shape[0]):
        eeg_data[k,:,:] = np.mean(x_train[y_train==k],axis=0)
    w_start = np.where(np.isin(f.times, [-0.5,0,0.4,1.7,2.3,3.8,4.3])==True)[0]  #select the few time window indicates
    rdms = []
    for n in range(len(w_start[:-1])): 
        rdms.append(np.mean(eeg_data[:, :, w_start[n]:w_start[n+1]], axis=2)) #not using similarity matrix just use raw data
    rdms_ss_v.append(rdms)

#perform PCA
dfs_mds=[]
categorical_model = list(product((1,2,3),(1,2,3)))
for i in range(len(w_start[:-1])):
    embedding = PCA(n_components=3)
    scaler = StandardScaler()
    a = [x[i] for x in rdms_ss_v]
    coor = embedding.fit_transform(scaler.fit_transform(np.delete(np.hstack(a), [1,4,7],axis=0))) #exclude the middle color values
    df = pd.DataFrame(np.hstack((coor,np.delete(categorical_model, [1,4,7],axis=0))), columns=['PC1','PC2','PC3','size','color'])
    df['time']=i
    dfs_mds.append(df)
dfs_mds = pd.concat(dfs_mds, ignore_index=True)

#plotting
fig, axs =plt.subplots(1,len(w_start[:-1]),figsize=(4*len(w_start[:-1])+2,4),sharex=True, sharey=True)
axs=axs.ravel()
palette=['g','r']
titles=['Pre-cue','Goal cue','Delay 1','Sample','Delay 2','Response']
dim1='PC1'; dim2='PC2'
feature='color' #depends on which 3 middle values were excluded
circularity_stim=[]
for i in range(len(w_start[:-1])):
    sns.scatterplot(x=dim1, y=dim2, data=dfs_mds[(dfs_mds.time==i)&(dfs_mds[feature]!=2)], size='size', hue='color',legend=False,sizes=[50,90,140],
                    edgecolor='k',palette=palette, ax=axs[i], zorder=2)
    c1x, c1y =[],[]
    for j,x in enumerate([0,2]):
        c1x.extend(dfs_mds[(dfs_mds.time==i)&(dfs_mds[feature]==x+1)][dim1].values)
        c1y.extend(dfs_mds[(dfs_mds.time==i)&(dfs_mds[feature]==x+1)][dim2].values)
    axs[i].plot([c1x[0],c1x[1],c1x[2],c1x[5],c1x[4],c1x[3],c1x[0]],[c1y[0],c1y[1],c1y[2],c1y[5],c1y[4],c1y[3],c1y[0]], 'gray',lw=1.4,zorder=1)
    axs[i].set_title(titles[i],fontsize=22)
    axs[i].set_xlabel('PC1',fontsize=16)
    axs[i].set_ylabel('PC2',fontsize=16)
    axs[i].set(xticklabels=[],yticklabels=[])
    axs[i].tick_params(bottom=False, left=False)
    polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[2],c1y[2]),(c1x[5],c1y[5]),(c1x[4],c1y[4]),(c1x[3],c1y[3]),(c1x[0],c1y[0])))
    circularity_stim.append(4*np.pi*polygon.area/(polygon.length)**2)
    c = 4*np.pi*polygon.area/(polygon.length)**2
sns.despine(fig=fig, right=True, top=True)

In [ ]:
#Figure 5A
#Load coherence data (weighted phase lag index ) and run statistics
con_file=[]
for subj in range(22):
    con_file.append(glob.glob('wpli_sub-%s.npy'%(subj+1))[0])
con_data = []
for con in con_file:
    con_data.append(np.load(con,allow_pickle=True))
#apply baseline
tfr = mne.time_frequency.EpochsTFR(reconst_epochs_p3.info, np.array(con_data), times, freqs)
tfr = tfr.apply_baseline(baseline=(-0.5,-0.3),mode='mean')
#Pick only coherence to posterior channels
X = np.transpose(tfr.copy().pick(picks_p).data.mean(1), (0,2,1)) #should be observations × time × frequencies × space
#run the cluster based permutation analysis
T_obs, clusters, p_values, _ = spatio_temporal_cluster_1samp_test(X, n_permutations=5000,threshold=1.96, tail=1,
                            n_jobs=1, buffer_size=2000,adjacency=None, max_step=1, verbose=1, out_type='indices')

#Plotting
times = tfr.times
# Create new stats image with only significant clusters
T_obs = tfr.copy().average().pick(picks_p).data.mean(0).T
T_obs_plot = np.nan * np.ones_like(T_obs)
for c, p_val in zip(clusters, p_values):
    if p_val <= 0.05:
        T_obs_plot[c] = T_obs[c]
        
vmax = np.max(np.abs(T_obs))
vmin = -vmax
fig, ax = plt.subplots(1, 1, figsize=(7,3.5))
plt.imshow(T_obs.T, cmap=plt.cm.gray,
           extent=[times[0], times[-1], freqs[0], freqs[-1]],
           aspect='auto', origin='lower', vmin=vmin, vmax=vmax)
plt.imshow(T_obs_plot.T, cmap=plt.cm.RdBu_r,
           extent=[times[0], times[-1], freqs[0], freqs[-1]],
           aspect='auto', origin='lower', vmin=vmin, vmax=vmax)
cb = plt.colorbar()
cb.ax.locator_params(nbins=5)
ax.set_yticks(freqs[0::2])
plt.axvline(0, linestyle='-', color='k', linewidth=1)
for y in [0.4,1.7,2.3,3.8]:
    plt.axvline(y, linestyle='--', color='k', linewidth=.8)
plt.ylabel('Frequency (Hz)', fontsize=15)
ax.set_xticks([0.2,1,2,3.1,4.1])
plt.title('Frontomedial theta to posterior coherence', fontsize=18)

In [ ]:
#Figure 5B and C
delay1_clu= clusters[np.where(p_values<0.05)[0][0]]#tuple(time indices x freqs indices)
#select delay time points: tfr.times==0 for 1st cluster, tfr.times==1.7 for second cluster
time_mask = delay1_clu[0]>= np.where(tfr.times==0.)[0] 

delay1_clu_x = delay1_clu[0][time_mask]
delay1_clu_y = delay1_clu[1][time_mask]
freq_mask = np.logical_and(delay1_clu_y>=np.where(tfr.freqs>=4)[0][0], 
                            delay1_clu_y<=np.where(tfr.freqs<7)[0][-1]) #select 1st theta freqs cluster
delay1_clu_x = delay1_clu_x[freq_mask]
delay1_clu_y = delay1_clu_y[freq_mask]
coh_ss = [x[(delay1_clu_x,delay1_clu_y)].mean() for x in X]
#print correlation between coherence strength and behavior
# for t in range(6):
#     circularity = dfs_mds_ss.loc[dfs_mds_ss.time==t].groupby(['subject'])['ci_%s%s'%(0,1)].mean().values 
#     print(pg.corr(circularity, coh_ss, method='spearman',alternative='greater'),'BF10', \
#           pg.corr(rankdata(circularity), rankdata(coh_ss), method='pearson',alternative='greater')['BF10'].values[0])

#Plotting for goal circularity
#Here you can select the task epoch to plot by passing the index for time
circularity =  dfs_mds_ss.loc[dfs_mds_ss.time==4].groupby(['subject'])['ci_%s%s'%(0,1)].mean()
fig, ax=plt.subplots(1,1,figsize=(5.5,4))
sns.regplot( coh_ss, circularity)
ax.set_ylabel('Circularity', fontsize=16)
ax.set_xlabel('FMT-to-posterior coherence', fontsize=16)
sns.despine(fig=fig,top=True, right=True)
ax.tick_params(bottom=False, left=True)
ax.set_xticks([-0.1,0,0.1,0.2])
ax.set_xticklabels([-0.1,0,0.1,0.2],fontsize=15)
ax.set_yticks([0,0.2,0.4,0.6])
ax.set_yticklabels([0,0.2,0.4,0.6],fontsize=15)

#stimulus circularity
delay1_clu= clusters[np.where(p_values<0.05)[0][0]]#tuple(time indices x freqs indices)
time_mask = delay1_clu[0]>= np.where(tfr.times==0.0)[0] #select delay time points
delay1_clu_x = delay1_clu[0][time_mask]
delay1_clu_y = delay1_clu[1][time_mask]
freq_mask = np.logical_and(delay1_clu_y>=np.where(tfr.freqs>=4)[0][0], 
                            delay1_clu_y<=np.where(tfr.freqs<7)[0][-1]) #select 1st theta freqs cluster
delay1_clu_x = delay1_clu_x[freq_mask]
delay1_clu_y = delay1_clu_y[freq_mask]
coh_ss = [x[(delay1_clu_x,delay1_clu_y)].mean() for x in X]
for t in range(6):
    circularity = dfs_mds_ss_stim.loc[dfs_mds_ss_stim.time==t].groupby(['subject'])['circular_index'].mean().values
    print(pg.corr(circularity, coh_ss, method='spearman',alternative='less'), 'BF10', \
          pg.corr(rankdata(circularity), rankdata(coh_ss), method='pearson',alternative='less')['BF10'].values[0])
print('\n')
for t in range(6):
    circularity = dfs_mds_ss_target.loc[dfs_mds_ss_target.time==t].groupby(['subject'])['circular_index'].mean().values
    print(pg.corr(circularity, coh_ss, method='spearman',alternative='greater'), 'BF10', \
          pg.corr(rankdata(circularity), rankdata(coh_ss), method='pearson',alternative='greater')['BF10'].values[0])

#Plotting scatter plot for stimulus circularity
circularity = dfs_mds_ss_stim.loc[dfs_mds_ss_stim.time==4].groupby(['subject'])['circular_index'].mean().values
fig, ax=plt.subplots(1,1,figsize=(5.5,4))
sns.regplot( coh_ss, circularity)
ax.set_ylabel('Circularity', fontsize=16)
ax.set_xticks([-0.1,0,0.1,0.2])
ax.set_xticklabels([-0.1,0,0.1,0.2],fontsize=15)
ax.set_yticks([0,0.2,0.4,0.6])
ax.set_yticklabels([0,0.2,0.4,0.6],fontsize=15)
ax.tick_params(bottom=False, left=True)
ax.set_xlabel('FMT-to-posterior coherence', fontsize=16)
sns.despine(fig=fig,top=True, right=True)

In [ ]:
#Figure 5D
delay1_clu= clusters[np.where(p_values<0.05)[0][0]]#tuple(time indices x freqs indices)
#select delay time points: tfr.times==0 for 1st cluster, tfr.times==1.7 for second cluster
time_mask = delay1_clu[0]>= np.where(tfr.times==1.7)[0] 
delay1_clu_x = delay1_clu[0][time_mask]
delay1_clu_y = delay1_clu[1][time_mask]
freq_mask = np.logical_and(delay1_clu_y>=np.where(tfr.freqs>=4)[0][0], 
                            delay1_clu_y<=np.where(tfr.freqs<7)[0][-1]) #select 1st theta freqs cluster
delay1_clu_x = delay1_clu_x[freq_mask]
delay1_clu_y = delay1_clu_y[freq_mask]
coh_ss = [x[(delay1_clu_x,delay1_clu_y)].mean() for x in X]

fig,ax = plt.subplots(1,1,figsize=(5,3))
clist=plt.cm.tab20([1,0])
#dfs_mds_ss is the individual correlation dataframe containing performance and circularity index
df = pd.DataFrame(np.concatenate([dfs_mds_ss.loc[dfs_mds_ss.time==0].groupby(['subject'])['error_total'].mean().values[np.where(coh_ss<np.percentile(coh_ss,50))[0]],
                                  dfs_mds_ss.loc[dfs_mds_ss.time==0].groupby(['subject'])['error_total'].mean().values[np.where(coh_ss>np.percentile(coh_ss,50))[0]]]),
                columns=['error'])
df['group']=np.repeat(['Low coherence','High coherence'],11)
df=pd.melt(df,id_vars='group',value_name='Response error')
sns.boxplot(data=df,x='group',y='Response error',width=0.4,palette=clist)
sns.despine(top=True,right=True)
_=ax.set_xticklabels(['Low coherence','High coherence'], fontsize=16,rotation=0)
ax.set_yticks([0.5,1,1.5])
ax.set_yticklabels([0.5,1,1.5],fontsize=15)
ax.set_ylabel('Response error',fontsize=16,wrap=True)

### FMRI experiment

In [ ]:
from nilearn import plotting, masking
from nilearn.image import mean_img, load_img, concat_imgs,index_img,binarize_img,math_img,clean_img, resample_to_img,smooth_img, threshold_img
import pandas as pd
import glob
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut, StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt
from nilearn.maskers import NiftiMasker
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from shapely.geometry import Polygon
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import copy
from itertools import product
import itertools
from joblib import Parallel, delayed
from nilearn.connectome import ConnectivityMeasure
from nilearn.maskers import MultiNiftiLabelsMasker

In [ ]:
def create_null(delay):
    inds = np.random.choice(50, len(subj_list))
    fnames = [glob.glob('s0%s_searchlight_goal_%s_smooth_shuffled_%d.nii.gz'%(sub, tr_, ind))[0] for sub, ind in zip(np.arange(1,22), inds)]
    data = mean_img(fnames).get_fdata()
    return data

In [ ]:
#Figure 6 
#To run searchlight analysis for circularity index
!python script_searchlight_circularity.py
#To generate null distribution for circularity searchlight, note this takes very long time
!python script_searchlight_circularity_null.py

In [ ]:
#From each iteration, draw an shuffled image per subject and calculate mean_img; repeat 10000 times
#The null data would be 10000 x n_voxel
#At each voxel calculate p values by comparing to the null distribution of mean
temp1 =  Parallel(n_jobs=30, prefer='processes')(delayed(create_null)('delay1') for i in range(10000))
temp2 =  Parallel(n_jobs=30, prefer='processes')(delayed(create_null)('delay2') for i in range(10000))
null_data=[]
null_data.append(np.array(temp1).reshape((len(temp1),-1)))
null_data.append(np.array(temp2).reshape((len(temp2),-1)))

p_imgs=[]
ref_img = "s01_delay1_smooth.nii.gz"
for j, tr_ in enumerate(['delay1','delay2']):
    data = mean_img([glob.glob('s0%s_searchlight_goal_%s_smooth.nii.gz'%(sub, tr_))[0] for sub in np.arange(1,22)]).get_fdata()
    p_array=np.zeros(data.flatten().shape)
    for i,v in enumerate(np.nonzero(data.flatten())[0]): #assign 1-pvalue to nonzero data (to save time)
        null = null_data[j][:,v] #get the null distribution at vth voxel
        p_array[v] = np.where(null <data.flatten()[v])[0].shape[0]/len(null)
    p_imgs.append(new_img_like(ref_img, p_array.reshape(data.shape)))

#Save the pvalue images somewhere, use software such as AFNI to threshold and visualize the result
p_imgs[0].to_filename('.nii.gz')
p_imgs[1].to_filename('.nii.gz')

In [ ]:
#Figure 6 group-level PCA visualization
categorical_model = [1,2,3,4]
scaler = StandardScaler()
ROI_mask = '' #This is a mask corresponding to a significant cluster from searchlight analysis
#Extract activity from the cluster
cluster_activity=[]
for sub in range(1,22):
    cluster_activity.append(masking.apply_mask('s0%s_%s_smooth.nii.gz'%(sub, tr_), ROI_mask, ensure_finite=True))

Xtrain=[]
for sub, BOLD in zip(np.arange(1,22), np.array(cluster_activity,dtype=object)):
    beha_fname = glob.glob('s0{a}_behavioural.csv'.format(a=sub))
    behavioural = pd.read_csv(beha_fname[0], delimiter=',',encoding_errors='ignore')
    #run-wise standardization
    fmri_masked =np.zeros(BOLD.shape)
    for sess in np.unique(behavioural['block_loop.thisN'].values):
        fmri_masked[behavioural['block_loop.thisN'].values==sess] = scaler.fit_transform(BOLD[behavioural['block_loop.thisN'].values==sess])
    y_train = behavioural['cue'].values
    x_train = np.zeros((np.unique(y_train).shape[0], BOLD.shape[1]))
    #prepare data matrix for PCA
    for i in range(np.unique(y_train).shape[0]):
        x_train[i,:] = np.mean(fmri_masked[y_train==i+1],axis=0)
    Xtrain.append(x_train)

#concatenate all subjects so that the resulting matrix is n_stimulus x (n_voxel*n_subj)
embedding2 = PCA(n_components=3)
coor = embedding2.fit_transform(standardscaler.fit_transform(np.hstack(Xtrain)))
dfs_mds = pd.DataFrame(coor, columns=['PC1','PC2','PC3'])
dfs_mds['cue'] = categorical_model 
sns.set_style('white')

palettes = ['yellow','green','r','darkblue']
circularity_xy=[]
dim1='PC1'; dim2='PC2'
#plotting
fig, ax =plt.subplots(1,1)
data=dfs_mds.groupby(['cue']).mean().reset_index()
c1x =data[dim1].values
c1y =data[dim2].values
ax.plot([c1x[0],c1x[1],c1x[3],c1x[2],c1x[0]],[c1y[0],c1y[1],c1y[3],c1y[2],c1y[0]],lw=1.4, c='gray', zorder=1)
sns.scatterplot(x=dim1, y=dim2, data=data, hue= 'cue',legend=False, ax=ax, palette=palettes, s=110,edgecolor='black',zorder=2, )
ax.set_xlabel('PC1',fontsize=16)
ax.set_ylabel('PC2',fontsize=16)
ax.set(xticklabels=[],yticklabels=[])
ax.tick_params(bottom=False, left=False)
polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[2],c1y[2]),(c1x[0],c1y[0])))
circularity_xy.append(4*np.pi*polygon.area/(polygon.length)**2)
c = 4*np.pi*polygon.area/(polygon.length)**2
ax.text(-30,10,r'$C = %.2f$'%c,fontsize=16)
sns.despine(fig=fig, right=True, top=True)

In [ ]:
#Figure 7
# grab center coordinates for atlas labels
#This file is a mask derived from combining selected clusters from multiple searchlight analyses (goal delay 1&2, stimulus and response)
coordinates = plotting.find_parcellation_cut_coords(labels_img='searchlight_ROI_combined.nii.gz') 
coordinates=np.delete(coordinates,[6,9],0)#we exclude the two ROIs with poor 2D geometry

masker = MultiNiftiLabelsMasker(
    labels_img='searchlight_ROI_combined.nii.gz',
    standardize="zscore",
    memory="nilearn_cache",
    detrend=False,
    n_jobs=20,
)

#Get time series of each cluster and calculate pairwise functional connectivity
time_series=[]
correlation_matrices=  []
mean_correlation_matrix=[]
for i in range(2):
    fname=[]
    for j,sub in enumerate(np.arange(1,22)):
        img= 's0%s_delay%d_smooth.nii.gz'%(sub, i+1)
        beha_fname = glob.glob('s0%s_behavioural.csv'%sub)
        behavioural = pd.read_csv(beha_fname[0], delimiter=',',encoding_errors='ignore')
        fname.append(clean_img(img, runs=behavioural['block_loop.thisN'].values, standardize=True, detrend=True))
    time_series.append(masker.fit_transform(fname))
    connectome_measure = ConnectivityMeasure(kind="correlation",standardize=False)
    # calculate correlation matrices across subjects
    correlation_matrices.append(connectome_measure.fit_transform(time_series[i]))
    # Mean correlation matrix across subjects can be grabbed like this,
    # using connectome measure object
    mean_correlation_matrix.append(connectome_measure.mean_)    


#7A
fig,ax = plt.subplots(2,1,figsize=(10,4), gridspec_kw={'height_ratios':[8,1]})
plotting.plot_connectome(
    np.zeros(data.shape),#Only plot the coordinates not connections so pass a zero array
    coordinates,
    node_size=30,
    node_color=['gold']*3+['blue']*3+['green']*2+['red']*2, #pass a customized color scheme (goal delay1, goal delay2, stimulus and response)
    edge_threshold=0.01,
    edge_vmax=0.4,
    colorbar=False,
    axes=ax[0],
    edge_cmap ='RdBu_r',
    annotate=True,
    alpha=0.85
)

#7B
sns.set_theme()
sns.set_style('white')
fig, axs = plt.subplots(1,3, figsize=(11,5), gridspec_kw={'width_ratios':[5,5,0.3]})
axs=axs.ravel()
labels= ['iPCS','left OFC','IFS','MTG','olPFC','mPFC','POS','ITG','left V4','right V4']
mask = np.triu(np.ones(10),1)
data = mean_correlation_matrix[0]
data=np.delete(data,[6,9],0) #we exclude the two ROIs with poor 2D geometry
data=np.delete(data,[6,9],1)
g1=sns.heatmap(data, xticklabels=labels, yticklabels=labels, annot=False,annot_kws={"size": 8},\
                    square=True, mask=mask,vmin=-0.3, vmax=0.3,center=0, cmap='RdBu_r',ax=axs[0],linewidths=0.5,cbar=False)
data = mean_correlation_matrix[1]
data=np.delete(data,[6,9],0) #we exclude the two ROIs with poor 2D geometry
data=np.delete(data,[6,9],1)
g2=sns.heatmap(data, xticklabels=labels, yticklabels='', annot=False,annot_kws={"size": 8}, \
                    square=True, mask=mask, vmin=-0.3,vmax=0.3,center=0,cmap='RdBu_r',ax=axs[1], linewidths=0.5,cbar_ax=axs[2])
g1.set_xticklabels(labels, rotation=25)
g2.set_xticklabels(labels, rotation=25)
sns.set(font_scale=1.2)             
# plt.show()

#run statistics and multiple comparison correction
cutoff=3
for i in range(2):
    ps=[]
    for x,y in itertools.combinations(np.arange(len(roi_names_sl)),2):
        if ~((x<cutoff and y<cutoff) or (x>=cutoff and y>=cutoff)):#don't include connections within clusters from the same delay periods
            matrix = correlation_matrices[i][:,x,y]
            result = pg.ttest(np.arctanh(matrix), 0)
            ps.append(result['p-val'])

    reject, ps_cor = pg.multicomp(ps,method = 'fdr_bh')#multiple comparison correction

    j=0
    for x,y in itertools.combinations(np.arange(len(roi_names_sl)),2):
        if ~((x<cutoff and y<cutoff) or (x>=cutoff and y>=cutoff)):
            if ~reject[j][0]:#set non significant correlation to zero
                mean_correlation_matrix[i][x,y] = 0
                mean_correlation_matrix[i][y,x] = 0 
            j+=1

    for x,y in itertools.combinations(np.arange(len(roi_names_sl)),2):
        if (x<cutoff and y<cutoff) or (x>=cutoff and y>=cutoff):
            mean_correlation_matrix[i][x,y] = 0
            mean_correlation_matrix[i][y,x] = 0
            
#7C-F
#Prepare the circularity and behavioural data
scaler = MinMaxScaler()
standardscaler=StandardScaler()
dfs_mds_ss =[]
for idx, (sub, xtrain) in enumerate(zip(np.arange(1,22), Xtrain)):
    beha_fname = glob.glob('s0%s_behavioural.csv'%sub)
    behavioural = pd.read_csv(beha_fname[0], delimiter=',',encoding_errors='ignore')
    behavioural ['error_color'] =  (behavioural.answer_color-behavioural.correct_color) #no within subject scaling, do it across subjects
    behavioural ['error_size'] = (behavioural.answer_size-behavioural.correct_size)
    behavioural ['error_size_per'] =behavioural.error_size/behavioural ['size'] #size is a percentile construt
    embedding =  PCA(n_components=2)
    coor = embedding.fit_transform(standardscaler.fit_transform(xtrain))
    df = pd.DataFrame(np.hstack((coor,categorical_model.reshape(-1,1))), columns=[0,1,'cue'])
    df['subject']=idx
    df['error_color'] = behavioural.error_color.abs().mean()
    df['error_size'] = behavioural.error_size.abs().mean()
    df['error_size_per'] = behavioural.error_size_per.abs().mean()
    dfs_mds_ss.append(df)

dfs_mds_ss = pd.concat(dfs_mds_ss, ignore_index=True) 
#scale the error terms across subjects before adding up for total error
dfs_mds_ss['error_color'] =  scaler.fit_transform(dfs_mds_ss.error_color.values.reshape(-1,1))
dfs_mds_ss['error_size_per'] =  scaler.fit_transform(dfs_mds_ss.error_size_per.values.reshape(-1,1))
dfs_mds_ss['error_total'] = dfs_mds_ss['error_color']+dfs_mds_ss['error_size_per']

for idx in range(21) :
    c1x =dfs_mds_ss[(dfs_mds_ss.subject==idx)][0].values
    c1y =dfs_mds_ss[(dfs_mds_ss.subject==idx)][1].values
    polygon = Polygon(((c1x[0],c1y[0]),(c1x[1],c1y[1]),(c1x[3],c1y[3]),(c1x[2],c1y[2]),(c1x[0],c1y[0])))
    dfs_mds_ss.loc[(dfs_mds_ss.subject==idx), 'ci'] = 4*np.pi*polygon.area/(polygon.length)**2

#Plot scatterplot between connection and behavioural performance or circularity index
#This is just an example for one of the scatterplots
ind1,ind2=0,8 #Select which connection to plot
delay=1 #select which delay
sns.set_style('white')
fig, ax=plt.subplots(1,1,figsize=(5.,4))
sns.regplot(x=correlation_matrices[delay][:,ind1,ind2],y= dfs_mds_ss.groupby(['subject']).error_total.mean())
ax.set_title('iPCS - ITG (Delay 2)', fontsize=17)
ax.set_xlabel('Connectivity strength', fontsize=17)
ax.set_ylabel('Response error', fontsize=17)
sns.despine(fig=fig,top=True, right=True)
ax.tick_params(bottom=True, left=True, labelsize=15)